In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sn
from scipy.interpolate import interp1d
import plotly.graph_objects as go

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder

In [2]:
wrld_timeseries_df = pd.read_csv('../optional_datasets/Novel_Corona_Virus_Dataset_Kaggle/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
wrld_timeseries_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/24/21,9/25/21,9/26/21,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,154960,154960,154960,155072,155093,155128,155174,155191,155191,155191
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,166690,167354,167893,168188,168782,169462,170131,170778,171327,171794
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,202449,202574,202722,202877,203045,203198,203359,203517,203657,203789
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,15167,15167,15167,15189,15192,15209,15222,15222,15222,15222
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,54280,54795,55121,55583,56040,56583,56583,58076,58603,58943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,736972,746678,756689,766051,770640,779398,790755,797712,803202,808578
275,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,394683,395677,396746,398946,400649,402255,403716,405056,405780,406652
276,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,8861,8891,8934,8988,9016,9039,9067,9111,9139,9139
277,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,208778,208829,208857,208867,208912,209002,209046,209114,209142,209163


In [3]:
wrld_timeseries_df.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '9/24/21', '9/25/21', '9/26/21', '9/27/21', '9/28/21', '9/29/21',
       '9/30/21', '10/1/21', '10/2/21', '10/3/21'],
      dtype='object', length=625)

In [4]:
wrld_timeseries_df.isna().sum()

Province/State    192
Country/Region      0
Lat                 2
Long                2
1/22/20             0
                 ... 
9/29/21             0
9/30/21             0
10/1/21             0
10/2/21             0
10/3/21             0
Length: 625, dtype: int64

In [5]:
#make new df and drop province/state and focus on country
country_timeseries= wrld_timeseries_df.drop(columns=["Province/State"])
country_timeseries

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,9/24/21,9/25/21,9/26/21,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21
0,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,0,...,154960,154960,154960,155072,155093,155128,155174,155191,155191,155191
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,166690,167354,167893,168188,168782,169462,170131,170778,171327,171794
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,202449,202574,202722,202877,203045,203198,203359,203517,203657,203789
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,15167,15167,15167,15189,15192,15209,15222,15222,15222,15222
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,54280,54795,55121,55583,56040,56583,56583,58076,58603,58943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,Vietnam,14.058324,108.277199,0,2,2,2,2,2,2,...,736972,746678,756689,766051,770640,779398,790755,797712,803202,808578
275,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,...,394683,395677,396746,398946,400649,402255,403716,405056,405780,406652
276,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,8861,8891,8934,8988,9016,9039,9067,9111,9139,9139
277,Zambia,-13.133897,27.849332,0,0,0,0,0,0,0,...,208778,208829,208857,208867,208912,209002,209046,209114,209142,209163


In [6]:
country_timeseries.isna().sum()

Country/Region    0
Lat               2
Long              2
1/22/20           0
1/23/20           0
                 ..
9/29/21           0
9/30/21           0
10/1/21           0
10/2/21           0
10/3/21           0
Length: 624, dtype: int64

In [7]:
country_timeseries.dropna(how='any', axis=0, inplace=True)
country_timeseries

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,9/24/21,9/25/21,9/26/21,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21
0,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,0,...,154960,154960,154960,155072,155093,155128,155174,155191,155191,155191
1,Albania,41.153300,20.168300,0,0,0,0,0,0,0,...,166690,167354,167893,168188,168782,169462,170131,170778,171327,171794
2,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,...,202449,202574,202722,202877,203045,203198,203359,203517,203657,203789
3,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,...,15167,15167,15167,15189,15192,15209,15222,15222,15222,15222
4,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,...,54280,54795,55121,55583,56040,56583,56583,58076,58603,58943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,Vietnam,14.058324,108.277199,0,2,2,2,2,2,2,...,736972,746678,756689,766051,770640,779398,790755,797712,803202,808578
275,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,...,394683,395677,396746,398946,400649,402255,403716,405056,405780,406652
276,Yemen,15.552727,48.516388,0,0,0,0,0,0,0,...,8861,8891,8934,8988,9016,9039,9067,9111,9139,9139
277,Zambia,-13.133897,27.849332,0,0,0,0,0,0,0,...,208778,208829,208857,208867,208912,209002,209046,209114,209142,209163


In [8]:
#double check null values
country_timeseries.isna().sum()

Country/Region    0
Lat               0
Long              0
1/22/20           0
1/23/20           0
                 ..
9/29/21           0
9/30/21           0
10/1/21           0
10/2/21           0
10/3/21           0
Length: 624, dtype: int64

In [9]:
#look for duplicated values
country_timeseries.duplicated().sum()

0

In [10]:
country_timeseries.describe()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,9/24/21,9/25/21,9/26/21,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21
count,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000,...,2.770000e+02,2.770000e+02,2.770000e+02,2.770000e+02,2.770000e+02,2.770000e+02,2.770000e+02,2.770000e+02,2.770000e+02,2.770000e+02
mean,20.305266,22.500100,2.010830,2.364621,3.397112,5.176895,7.646209,10.566787,20.137184,22.263538,...,8.344931e+05,8.358054e+05,8.370003e+05,8.387620e+05,8.403761e+05,8.421893e+05,8.439403e+05,8.459058e+05,8.471392e+05,8.481936e+05
std,25.206138,75.408236,26.733451,26.831207,33.404109,46.491792,64.973443,87.542330,214.814639,216.133076,...,3.659407e+06,3.663315e+06,3.666382e+06,3.675692e+06,3.681969e+06,3.689057e+06,3.695931e+06,3.704504e+06,3.707895e+06,3.710510e+06
min,-51.796300,-178.116500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.860416,-23.041800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.642000e+03,2.642000e+03,2.642000e+03,2.642000e+03,2.642000e+03,2.642000e+03,2.642000e+03,2.854000e+03,2.859000e+03,2.868000e+03
50%,21.521757,20.939400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.735500e+04,3.740000e+04,3.744800e+04,3.747200e+04,3.750100e+04,3.753500e+04,3.828300e+04,3.828300e+04,3.828300e+04,3.828300e+04
75%,40.463667,85.240100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.630170e+05,3.630170e+05,3.630170e+05,3.633000e+05,3.661500e+05,3.676720e+05,3.689680e+05,3.689680e+05,3.703680e+05,3.733320e+05
max,71.706900,178.065000,444.000000,444.000000,549.000000,761.000000,1058.000000,1423.000000,3554.000000,3554.000000,...,4.285337e+07,4.290090e+07,4.293186e+07,4.311688e+07,4.322648e+07,4.334975e+07,4.346034e+07,4.361863e+07,4.365783e+07,4.368305e+07


In [11]:
#set index to country
country_timeseries= country_timeseries.set_index('Country/Region')
country_timeseries.head()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,9/24/21,9/25/21,9/26/21,9/27/21,9/28/21,9/29/21,9/30/21,10/1/21,10/2/21,10/3/21
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,...,154960,154960,154960,155072,155093,155128,155174,155191,155191,155191
Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,...,166690,167354,167893,168188,168782,169462,170131,170778,171327,171794
Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,...,202449,202574,202722,202877,203045,203198,203359,203517,203657,203789
Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,...,15167,15167,15167,15189,15192,15209,15222,15222,15222,15222
Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,...,54280,54795,55121,55583,56040,56583,56583,58076,58603,58943


In [12]:
country_timeseries.dtypes

Lat        float64
Long       float64
1/22/20      int64
1/23/20      int64
1/24/20      int64
            ...   
9/29/21      int64
9/30/21      int64
10/1/21      int64
10/2/21      int64
10/3/21      int64
Length: 623, dtype: object

In [13]:
#goupby country 
cntry_confirmed= country_timeseries.groupby(['Country/Region']).sum()
#look at top 20 countries
n = 20
cntry_confirmed = cntry_confirmed.sort_values(by=country_timeseries.columns[-1],\
                                                          ascending=False)[0:n]
cntry_confirmed = cntry_confirmed.drop(['Lat', 'Long'], axis=1)

In [19]:
# change the pop. to read in millions 
scale_pop = 10000000  # scale population number by millions
population = []
df_pop = pd.read_csv('../optional_datasets/Novel_Corona_Virus_Dataset_Kaggle/population_by_country_2020.csv')
for j in range(n):
    country = cntry_confirmed.index[j]
    population.append((df_pop[df_pop['Country (or dependency)'] == country]\
                       ['Population (2020)'] / scale_pop).to_numpy())

In [20]:
countries=cntry_confirmed .index
cumulative = cntry_confirmed .iloc[:,-1]

fig = go.Figure(data=[
    go.Bar(name='Total Cases', x=countries, y=cumulative),
    go.Bar(name='Normalized', x=countries, y=cumulative / population )
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_yaxes(type='log')

fig.update_layout(title="Countries with most Covid Cases<br>Total Confirmed Cases and<br>Total Confirmed Cases Normalized per Million in Population",
                title_x=0.5,
                  yaxis_title="Cases - Cases Normalized")
fig.show()

/Users/celestemoreno/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/computation/expressions.py:69: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

